In [1]:
%matplotlib inline
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
from sklearn import datasets
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import RFE
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
import statsmodels.api as sm

In [2]:
# Load the data
# Statistics of each game for 10 seasons
file_path = './epl_game_data/2021_2022_season.csv'
raw_df = pd.read_csv(file_path)
raw_df

,Unnamed: 0,Season,DateTime,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,...,HST,AST,HC,AC,HF,AF,HY,AY,HR,AR
0,0,2021-22,2021-08-13T20:00:00Z,Brentford,Arsenal,2,0,H,1.0,0.0,...,3.0,4.0,2.0,5.0,12.0,8.0,0.0,0.0,0.0,0.0
1,1,2021-22,2021-08-14T12:30:00Z,Man United,Leeds,5,1,H,1.0,0.0,...,8.0,3.0,5.0,4.0,11.0,9.0,1.0,2.0,0.0,0.0
2,2,2021-22,2021-08-14T15:00:00Z,Burnley,Brighton,1,2,A,1.0,0.0,...,3.0,8.0,7.0,6.0,10.0,7.0,2.0,1.0,0.0,0.0
3,3,2021-22,2021-08-14T15:00:00Z,Chelsea,Crystal Palace,3,0,H,2.0,0.0,...,6.0,1.0,5.0,2.0,15.0,11.0,0.0,0.0,0.0,0.0
4,4,2021-22,2021-08-14T15:00:00Z,Everton,Southampton,3,1,H,0.0,1.0,...,6.0,3.0,6.0,8.0,13.0,15.0,2.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
304,304,2021-22,2022-04-09T17:30:00Z,Aston Villa,Tottenham,0,4,A,0.0,1.0,...,8.0,5.0,9.0,3.0,12.0,14.0,2.0,3.0,0.0,0.0
305,305,2021-22,2022-04-10T14:00:00Z,Brentford,West Ham,2,0,H,0.0,0.0,...,7.0,1.0,4.0,6.0,2.0,6.0,0.0,1.0,0.0,0.0
306,306,2021-22,2022-04-10T14:00:00Z,Leicester,Crystal Palace,2,1,H,2.0,0.0,...,3.0,3.0,3.0,4.0,11.0,12.0,1.0,1.0,0.0,0.0
307,307,2021-22,2022-04-10T14:00:00Z,Norwich,Burnley,2,0,H,1.0,0.0,...,6.0,4.0,6.0,7.0,12.0,10.0,1.0,1.0,0.0,0.0


In [3]:
# Function to transform Full Time Results (FTR) into numeric data type
def transformResult(row):
    if(row.FTR == 'H'):
        return 1
    elif(row.FTR == 'A'):
        return -1
    else:
        return 0

raw_df['Result'] = raw_df.apply(lambda row: transformResult(row),axis=1)

# Check that it worked.
raw_df

,Unnamed: 0,Season,DateTime,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,...,AST,HC,AC,HF,AF,HY,AY,HR,AR,Result
0,0,2021-22,2021-08-13T20:00:00Z,Brentford,Arsenal,2,0,H,1.0,0.0,...,4.0,2.0,5.0,12.0,8.0,0.0,0.0,0.0,0.0,1
1,1,2021-22,2021-08-14T12:30:00Z,Man United,Leeds,5,1,H,1.0,0.0,...,3.0,5.0,4.0,11.0,9.0,1.0,2.0,0.0,0.0,1
2,2,2021-22,2021-08-14T15:00:00Z,Burnley,Brighton,1,2,A,1.0,0.0,...,8.0,7.0,6.0,10.0,7.0,2.0,1.0,0.0,0.0,-1
3,3,2021-22,2021-08-14T15:00:00Z,Chelsea,Crystal Palace,3,0,H,2.0,0.0,...,1.0,5.0,2.0,15.0,11.0,0.0,0.0,0.0,0.0,1
4,4,2021-22,2021-08-14T15:00:00Z,Everton,Southampton,3,1,H,0.0,1.0,...,3.0,6.0,8.0,13.0,15.0,2.0,0.0,0.0,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
304,304,2021-22,2022-04-09T17:30:00Z,Aston Villa,Tottenham,0,4,A,0.0,1.0,...,5.0,9.0,3.0,12.0,14.0,2.0,3.0,0.0,0.0,-1
305,305,2021-22,2022-04-10T14:00:00Z,Brentford,West Ham,2,0,H,0.0,0.0,...,1.0,4.0,6.0,2.0,6.0,0.0,1.0,0.0,0.0,1
306,306,2021-22,2022-04-10T14:00:00Z,Leicester,Crystal Palace,2,1,H,2.0,0.0,...,3.0,3.0,4.0,11.0,12.0,1.0,1.0,0.0,0.0,1
307,307,2021-22,2022-04-10T14:00:00Z,Norwich,Burnley,2,0,H,1.0,0.0,...,4.0,6.0,7.0,12.0,10.0,1.0,1.0,0.0,0.0,1


In [4]:
# Get a list of all the column names.
raw_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 309 entries, 0 to 308
Data columns (total 25 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  309 non-null    int64  
 1   Season      309 non-null    object 
 2   DateTime    309 non-null    object 
 3   HomeTeam    309 non-null    object 
 4   AwayTeam    309 non-null    object 
 5   FTHG        309 non-null    int64  
 6   FTAG        309 non-null    int64  
 7   FTR         309 non-null    object 
 8   HTHG        309 non-null    float64
 9   HTAG        309 non-null    float64
 10  HTR         309 non-null    object 
 11  Referee     309 non-null    object 
 12  HS          309 non-null    float64
 13  AS          309 non-null    float64
 14  HST         309 non-null    float64
 15  AST         309 non-null    float64
 16  HC          309 non-null    float64
 17  AC          309 non-null    float64
 18  HF          309 non-null    float64
 19  AF          309 non-null    f

In [5]:
# Get each team's offensive/defensive capabilities per season.
def get_features_per_season(data_now):
    table_new = pd.DataFrame(columns=('Team','HAS','AAS'
                                      ,'HAC','AAC'
                                      ,'HtHAS','HtAAS'
                                      ,'HtHAC','HtAAC'
                                      ,'HAST','AAST'))
    
    home_data = data_now.groupby('HomeTeam')
    away_data = data_now.groupby('AwayTeam')
    
    # Sum the full time home and away goals.
    table_new.Team = list(home_data.groups.keys())
    home_goals_scored = home_data.FTHG.sum().values
    home_goals_conceded = home_data.FTAG.sum().values
    away_goals_scored = away_data.FTAG.sum().values
    away_goals_conceded = away_data.FTHG.sum().values
    
    # Sum the half time home and away goals.
    home_halftime_goals_scored = home_data.HTHG.sum().values
    home_halftime_goals_conceded = home_data.HTAG.sum().values
    away_halftime_goals_scored = away_data.HTAG.sum().values
    away_halftime_goals_conceded = away_data.HTHG.sum().values
    
    # Sum the shots on target
    home_shots_on_target = home_data.HST.sum().values
    away_shots_on_target = home_data.AST.sum().values
    
    
    # Average full time home/away attacking/defensive strength.
    table_new.HAS = (home_goals_scored / 19.0)  # avg home goals scored
    table_new.AAS = (away_goals_scored / 19.0)  # avg away goals scored
    table_new.HAC = (home_goals_conceded / 19.0)  # avg home goals conceded
    table_new.AAC = (away_goals_conceded / 19.0)  # avg away goals conceded
    
    # Average half time home/away attacking/defensive strength.
    table_new.HtHAS = (home_halftime_goals_scored / 19.0) # avg half-time home goals scored
    table_new.HtHAC = (away_halftime_goals_scored / 19.0) # avg half-time away goals scored
    table_new.HtAAS = (home_halftime_goals_conceded / 19.0) # avg half-time home goals conceded
    table_new.HtAAC = (away_halftime_goals_conceded / 19.0) # avg half-time away goals conceded
    
    # Average the shots on target.
    table_new.HAST = (home_shots_on_target / 19.0) # avg half-time home goals scored
    table_new.AAST = (away_shots_on_target / 19.0) # avg half-time away goals scored
  
    
    return table_new

# create a team dataframe using our function to calculate strengths and averages for home and away.
team_data = get_features_per_season(raw_df)
# Set the team as the index (this is used to merge dataframes in the next step).
team_data.set_index('Team', inplace=True)
# Create an empty team dataframe to use at the end, in order to get a final Premier League table.
last_season_table = pd.DataFrame(index=team_data.index)
# Scale the data.
scaler = MinMaxScaler()
# Fit the data to the scaler and show the final form.
team_data = pd.DataFrame(scaler.fit_transform(team_data), columns=team_data.columns, index=team_data.index)
team_data

,HAS,AAS,HAC,AAC,HtHAS,HtAAS,HtHAC,HtAAC,HAST,AAST
Team,,,,,,,,,,
Arsenal,0.46875,0.37500,0.233333,0.466667,0.352941,0.083333,0.666667,0.5000,0.626866,0.218182
Aston Villa,0.43750,0.31250,0.600000,0.433333,0.352941,0.750000,0.222222,0.6250,0.208955,0.454545
Brentford,0.25000,0.40625,0.400000,0.700000,0.000000,0.583333,0.333333,0.9375,0.238806,0.454545
Brighton,0.00000,0.31250,0.433333,0.300000,0.117647,0.666667,0.166667,0.5000,0.000000,0.581818
Burnley,0.09375,0.12500,0.433333,0.533333,0.176471,0.583333,0.222222,0.6875,0.029851,0.563636
Chelsea,0.59375,0.84375,0.233333,0.033333,0.470588,0.000000,0.833333,0.1875,0.567164,0.218182
Crystal Palace,0.46875,0.31250,0.333333,0.500000,0.294118,0.583333,0.333333,0.7500,0.223881,0.345455
Everton,0.31250,0.15625,0.400000,0.833333,0.117647,0.416667,0.166667,0.8750,0.134328,0.454545
Leeds,0.25000,0.37500,0.766667,1.000000,0.235294,0.833333,0.333333,1.0000,0.283582,1.000000


In [6]:
# Create a function that ties the HomeAttackingStrength(HAS), AwayAttackingStrenght(AAS),
# HomeDefensiveStrength(HDS), AwayDefensiveStrength(ADS) to each game for the 2019/20 season.
h_a_cols = ['HomeTeam', 'AwayTeam']
data_prep = raw_df[h_a_cols]

# Add in the Strengths by Home and Away teams.
data_prep = data_prep.merge(team_data['HAS'], left_on='HomeTeam', right_index=True, how='left')
data_prep = data_prep.merge(team_data['HAC'], left_on='HomeTeam', right_index=True, how='left')
data_prep = data_prep.merge(team_data['AAS'], left_on='AwayTeam', right_index=True, how='left')
data_prep = data_prep.merge(team_data['AAC'], left_on='AwayTeam', right_index=True, how='left')

# Add in the half-time strengths by home and away teams.
# data_prep = data_prep.merge(team_data['HtHAS'], left_on='HomeTeam', right_index=True, how='left')
# data_prep = data_prep.merge(team_data['HtHAC'], left_on='HomeTeam', right_index=True, how='left')
# data_prep = data_prep.merge(team_data['HtAAS'], left_on='AwayTeam', right_index=True, how='left')
# data_prep = data_prep.merge(team_data['HtAAC'], left_on='AwayTeam', right_index=True, how='left')


# Add in the average shots on target.
# data_prep = data_prep.merge(team_data['HAST'], left_on='AwayTeam', right_index=True, how='left')
# data_prep = data_prep.merge(team_data['AAST'], left_on='AwayTeam', right_index=True, how='left')


# Calculate the attacking strengths of each team
data_prep['HAtS'] = (data_prep['HAS'] * data_prep['AAC']); # Home Attack Strength
data_prep['AAtS'] = (data_prep['HAC'] * data_prep['AAS']); # Away Attack Strength
    
    
# Add in the Results
data_prep = data_prep.merge(raw_df['Result'], left_index=True, right_index=True)

data_prep

,HomeTeam,AwayTeam,HAS,HAC,AAS,AAC,HAtS,AAtS,Result
0,Brentford,Arsenal,0.25000,0.400000,0.37500,0.466667,0.116667,0.150000,1
1,Man United,Leeds,0.46875,0.400000,0.37500,1.000000,0.468750,0.150000,1
2,Burnley,Brighton,0.09375,0.433333,0.31250,0.300000,0.028125,0.135417,-1
3,Chelsea,Crystal Palace,0.59375,0.233333,0.31250,0.500000,0.296875,0.072917,1
4,Everton,Southampton,0.31250,0.400000,0.28125,0.800000,0.250000,0.112500,1
...,...,...,...,...,...,...,...,...,...
304,Aston Villa,Tottenham,0.43750,0.600000,0.53125,0.400000,0.175000,0.318750,-1
305,Brentford,West Ham,0.25000,0.400000,0.43750,0.433333,0.108333,0.175000,1
306,Leicester,Crystal Palace,0.50000,0.433333,0.31250,0.500000,0.250000,0.135417,1
307,Norwich,Burnley,0.06250,0.800000,0.12500,0.533333,0.033333,0.100000,1


In [7]:
# Check that there are no null values so we can ensure we are using a complete dataset.
data_prep.isnull().sum()

HomeTeam    0
AwayTeam    0
HAS         0
HAC         0
AAS         0
AAC         0
HAtS        0
AAtS        0
Result      0
dtype: int64

In [8]:
# Turn our game dataframe into a numerical dataframe by dropping the string columns.
rm_col_list = ['HomeTeam','AwayTeam']
col_list = data_prep.columns.values.tolist()
col_list = [x for x in col_list if x not in rm_col_list]

to_use = data_prep[col_list]
X = [x for x in col_list if x != 'Result']
Y = [x for x in col_list if x == 'Result']

to_use

,HAS,HAC,AAS,AAC,HAtS,AAtS,Result
0,0.25000,0.400000,0.37500,0.466667,0.116667,0.150000,1
1,0.46875,0.400000,0.37500,1.000000,0.468750,0.150000,1
2,0.09375,0.433333,0.31250,0.300000,0.028125,0.135417,-1
3,0.59375,0.233333,0.31250,0.500000,0.296875,0.072917,1
4,0.31250,0.400000,0.28125,0.800000,0.250000,0.112500,1
...,...,...,...,...,...,...,...
304,0.43750,0.600000,0.53125,0.400000,0.175000,0.318750,-1
305,0.25000,0.400000,0.43750,0.433333,0.108333,0.175000,1
306,0.50000,0.433333,0.31250,0.500000,0.250000,0.135417,1
307,0.06250,0.800000,0.12500,0.533333,0.033333,0.100000,1


In [9]:
# Seperate our data into X and Y sections and instantiate, fit and get a score of 
# the first model
X_data = to_use.drop('Result', axis=1)
Y_data = to_use['Result']

model = RandomForestClassifier()
model.fit(X_data, Y_data)
model.score(X_data, Y_data)

1.0

In [10]:
# Seperate our data into training and testing and instantiate our second model.
X_train, X_test, Y_train, Y_test = train_test_split(X_data, Y_data, test_size=0.2, random_state=42)

model1 = RandomForestClassifier()
model1.fit(X_train, Y_train)

RandomForestClassifier()

In [11]:
# Get the predicted scores into from our model.
predicted = model1.predict(X_test)
predicted

array([ 1,  0, -1,  1,  1,  1,  1, -1, -1,  1,  1,  1,  1,  1,  1, -1, -1,
        0,  1, -1,  1, -1, -1, -1, -1, -1,  0, -1,  1,  1,  0,  1,  1,  0,
        1, -1,  0, -1,  1, -1,  0, -1, -1, -1, -1,  0,  1, -1,  1,  1,  0,
        1,  0, -1, -1,  1,  0,  0, -1,  1,  0,  0])

In [12]:
# Get the training and test accuracy/variance scores of our models and assess. 
acc_score = accuracy_score(Y_test, predicted)
print("Accuracy score: " + str(acc_score))
scores = cross_val_score(model1, X_train, Y_train, cv=10)
print("Cross val scores")
print(scores)

Accuracy score: 0.5161290322580645
Cross val scores
[0.32       0.6        0.6        0.56       0.68       0.44
 0.56       0.5        0.5        0.41666667]


In [13]:
score_train = model1.score(X_train, Y_train)
score_test = model1.score(X_test, Y_test)

print("Training set accuracy: ", '%.3f'%(score_train))
print("Test set accuracy: ", '%.3f'%(score_test))

Training set accuracy:  1.000
Test set accuracy:  0.516


In [14]:
# Turn the predicted array into a dataframe.
predicted = model1.predict(X_data)
predicted
predictedDF = pd.DataFrame(predicted, columns=['Predicted'])
predictedDF.head()

,Predicted
0,1
1,1
2,-1
3,1
4,1


In [15]:
# Merge the games dataframe with actual and predicted results.
to_use = to_use.merge(predictedDF, left_index=True, right_index=True)
fixture_list = data_prep.merge(predictedDF, left_index=True, right_index=True)
fixture_list

,HomeTeam,AwayTeam,HAS,HAC,AAS,AAC,HAtS,AAtS,Result,Predicted
0,Brentford,Arsenal,0.25000,0.400000,0.37500,0.466667,0.116667,0.150000,1,1
1,Man United,Leeds,0.46875,0.400000,0.37500,1.000000,0.468750,0.150000,1,1
2,Burnley,Brighton,0.09375,0.433333,0.31250,0.300000,0.028125,0.135417,-1,-1
3,Chelsea,Crystal Palace,0.59375,0.233333,0.31250,0.500000,0.296875,0.072917,1,1
4,Everton,Southampton,0.31250,0.400000,0.28125,0.800000,0.250000,0.112500,1,1
...,...,...,...,...,...,...,...,...,...,...
304,Aston Villa,Tottenham,0.43750,0.600000,0.53125,0.400000,0.175000,0.318750,-1,-1
305,Brentford,West Ham,0.25000,0.400000,0.43750,0.433333,0.108333,0.175000,1,1
306,Leicester,Crystal Palace,0.50000,0.433333,0.31250,0.500000,0.250000,0.135417,1,1
307,Norwich,Burnley,0.06250,0.800000,0.12500,0.533333,0.033333,0.100000,1,1


In [16]:
# Calculate points based off of wins, draws, and losses.
last_season_table['Actual Points'] = 0 
last_season_table['Predicted Points'] = 0 

def make_actual_table(x, ht, at, which):
    if x == 1:
        last_season_table.loc[ht, which] += 3
    elif x == 0:
        last_season_table.loc[ht, which] += 1
        last_season_table.loc[at, which] += 1
    elif x == -1:
        last_season_table.loc[at, which] += 3

fixture_list.apply(lambda x: make_actual_table(x.Result, x.HomeTeam, x.AwayTeam, 'Actual Points'), axis=1)
fixture_list.apply(lambda x: make_actual_table(x.Predicted, x.HomeTeam, x.AwayTeam, 'Predicted Points'), axis=1)


0      None
1      None
2      None
3      None
4      None
       ... 
304    None
305    None
306    None
307    None
308    None
Length: 309, dtype: object

In [17]:
# 2021/22 Season actual vs. predicted table
actual_index = last_season_table.sort_values(by='Actual Points', ascending=0).index
predicted_index = last_season_table.sort_values(by='Predicted Points', ascending=0).index

last_season_table.sort_values(by='Actual Points', ascending=0)

,Actual Points,Predicted Points
Team,,
Man City,74,78
Liverpool,73,80
Chelsea,62,62
Tottenham,57,51
Arsenal,54,58
Man United,51,51
West Ham,51,50
Wolves,49,60
Leicester,40,34


In [18]:
# Getting data for the newly promoted teams.
# Load the new season
path_ = './epl_game_data/epl-2022-UTC.csv'
new_season_df = pd.read_csv(path_)
new_season_df.head(20)

,Match Number,Round Number,Date,Location,Home Team,Away Team,Result
0,1,1,05/08/2022 19:00,Selhurst Park,Crystal Palace,Arsenal,0 - 2
1,2,1,06/08/2022 11:30,Craven Cottage,Fulham,Liverpool,2 - 2
2,3,1,06/08/2022 14:00,Vitality Stadium,Bournemouth,Aston Villa,2 - 0
3,4,1,06/08/2022 14:00,Elland Road,Leeds,Wolves,2 - 1
4,6,1,06/08/2022 14:00,St. James' Park,Newcastle,Nottingham Forest,2 - 0
5,7,1,06/08/2022 14:00,Tottenham Hotspur Stadium,Spurs,Southampton,4 - 1
6,8,1,06/08/2022 16:30,Goodison Park,Everton,Chelsea,0 - 1
7,5,1,07/08/2022 13:00,King Power Stadium,Leicester,Brentford,2 - 2
8,9,1,07/08/2022 13:00,Old Trafford,Man Utd,Brighton,1 - 2
9,10,1,07/08/2022 15:30,London Stadium,West Ham,Man City,0 - 2


In [19]:
# Correct team names: Man Utd to Manchester United, Spurs to Tottenham, 
corrected_names = {
    'Man Utd': 'Man United',
    'Spurs': 'Tottenham'
}

new_season_df.replace({'Home Team':corrected_names,'Away Team':corrected_names},inplace=True)
new_season_df.head(20)

,Match Number,Round Number,Date,Location,Home Team,Away Team,Result
0,1,1,05/08/2022 19:00,Selhurst Park,Crystal Palace,Arsenal,0 - 2
1,2,1,06/08/2022 11:30,Craven Cottage,Fulham,Liverpool,2 - 2
2,3,1,06/08/2022 14:00,Vitality Stadium,Bournemouth,Aston Villa,2 - 0
3,4,1,06/08/2022 14:00,Elland Road,Leeds,Wolves,2 - 1
4,6,1,06/08/2022 14:00,St. James' Park,Newcastle,Nottingham Forest,2 - 0
5,7,1,06/08/2022 14:00,Tottenham Hotspur Stadium,Tottenham,Southampton,4 - 1
6,8,1,06/08/2022 16:30,Goodison Park,Everton,Chelsea,0 - 1
7,5,1,07/08/2022 13:00,King Power Stadium,Leicester,Brentford,2 - 2
8,9,1,07/08/2022 13:00,Old Trafford,Man United,Brighton,1 - 2
9,10,1,07/08/2022 15:30,London Stadium,West Ham,Man City,0 - 2


In [20]:
# Drop Match Number and Round Number and check remianing data types.
new_team_scores = new_season_df
new_team_scores.drop(['Match Number','Round Number', 'Location'],axis=1, inplace=True)
new_team_scores = new_team_scores.dropna(axis=0, how='any')
new_team_scores.head()

,Date,Home Team,Away Team,Result
0,05/08/2022 19:00,Crystal Palace,Arsenal,0 - 2
1,06/08/2022 11:30,Fulham,Liverpool,2 - 2
2,06/08/2022 14:00,Bournemouth,Aston Villa,2 - 0
3,06/08/2022 14:00,Leeds,Wolves,2 - 1
4,06/08/2022 14:00,Newcastle,Nottingham Forest,2 - 0


In [21]:
# Seperate the results column into Full Time Home Goals and Away Time Home Goals
new_team_scores['FTHG'] = new_team_scores['Result'].astype(str).str[0]
new_team_scores['FTAG'] = new_team_scores['Result'].str.strip().str[-1]
new_team_scores.drop('Result',axis=1, inplace=True)
new_team_scores

/Users/samuelcaslowitz/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/samuelcaslowitz/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/samuelcaslowitz/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value i

,Date,Home Team,Away Team,FTHG,FTAG
0,05/08/2022 19:00,Crystal Palace,Arsenal,0,2
1,06/08/2022 11:30,Fulham,Liverpool,2,2
2,06/08/2022 14:00,Bournemouth,Aston Villa,2,0
3,06/08/2022 14:00,Leeds,Wolves,2,1
4,06/08/2022 14:00,Newcastle,Nottingham Forest,2,0
5,06/08/2022 14:00,Tottenham,Southampton,4,1
6,06/08/2022 16:30,Everton,Chelsea,0,1
7,07/08/2022 13:00,Leicester,Brentford,2,2
8,07/08/2022 13:00,Man United,Brighton,1,2
9,07/08/2022 15:30,West Ham,Man City,0,2


In [22]:
# Check the data types.
new_team_scores.dtypes

Date         object
Home Team    object
Away Team    object
FTHG         object
FTAG         object
dtype: object

In [23]:
# In order to transform our dataset, we will need to turn the FTHG and FTAG into integers
new_team_scores['FTHG'] = pd.to_numeric(new_team_scores['FTHG'])
new_team_scores['FTAG'] = pd.to_numeric(new_team_scores['FTAG'])
new_team_scores.dtypes

/Users/samuelcaslowitz/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/samuelcaslowitz/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


Date         object
Home Team    object
Away Team    object
FTHG          int64
FTAG          int64
dtype: object

In [24]:
curr_home_data = new_team_scores.groupby('Home Team').sum()
curr_away_data = new_team_scores.groupby('Away Team').sum()
curr_home_data['HAS'] = curr_home_data['FTHG'] / 19 # Home Average Scores
curr_home_data['HAC'] = curr_home_data['FTAG'] / 19 # Home Average Conceded
curr_away_data['AAS'] = curr_away_data['FTAG'] / 19 # Away average Scored
curr_away_data['AAC'] = curr_away_data['FTHG'] / 19 # Away Average Conceded
curr_team_data = pd.DataFrame()
curr_team_data['Team'] = curr_home_data.index

curr_team_data = curr_team_data.merge(curr_home_data['HAS'], left_on='Team', right_index=True)
curr_team_data = curr_team_data.merge(curr_home_data['HAC'], left_on='Team', right_index=True)

curr_team_data = curr_team_data.merge(curr_away_data['AAS'], left_on='Team', right_index=True)
curr_team_data = curr_team_data.merge(curr_away_data['AAC'], left_on='Team', right_index=True)

curr_team_data.set_index("Team", inplace=True)
curr_season_table= pd.DataFrame(index=curr_team_data.index)
curr_team_data = team_data.merge(curr_team_data, 
                                 left_index=True, right_index=True,how='outer'
                                )
curr_team_data = pd.DataFrame(scaler.fit_transform(curr_team_data), columns=curr_team_data.columns, index=curr_team_data.index)
curr_team_data

,HAS_x,AAS_x,HAC_x,AAC_x,HtHAS,HtAAS,HtHAC,HtAAC,HAST,AAST,HAS_y,HAC_y,AAS_y,AAC_y
Team,,,,,,,,,,,,,,
Arsenal,0.46875,0.37500,0.233333,0.466667,0.352941,0.083333,0.666667,0.5000,0.626866,0.218182,0.538462,0.75,1.0,0.000000
Aston Villa,0.43750,0.31250,0.600000,0.433333,0.352941,0.750000,0.222222,0.6250,0.208955,0.454545,0.153846,0.50,0.2,0.333333
Bournemouth,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.076923,0.50,0.4,1.000000
Brentford,0.25000,0.40625,0.400000,0.700000,0.000000,0.583333,0.333333,0.9375,0.238806,0.454545,0.692308,0.50,0.8,0.250000
Brighton,0.00000,0.31250,0.433333,0.300000,0.117647,0.666667,0.166667,0.5000,0.000000,0.581818,0.384615,0.25,0.8,0.000000
Burnley,0.09375,0.12500,0.433333,0.533333,0.176471,0.583333,0.222222,0.6875,0.029851,0.563636,NaN,NaN,NaN,NaN
Chelsea,0.59375,0.84375,0.233333,0.033333,0.470588,0.000000,0.833333,0.1875,0.567164,0.218182,0.384615,0.75,0.2,0.166667
Crystal Palace,0.46875,0.31250,0.333333,0.500000,0.294118,0.583333,0.333333,0.7500,0.223881,0.345455,0.230769,0.75,0.4,0.166667
Everton,0.31250,0.15625,0.400000,0.833333,0.117647,0.416667,0.166667,0.8750,0.134328,0.454545,0.000000,0.25,0.4,0.083333


In [25]:
# Drop the teams that were relegated last season.
relegated = ['Burnley','Norwich','Watford']
curr_team_data.drop(index=relegated)


,HAS_x,AAS_x,HAC_x,AAC_x,HtHAS,HtAAS,HtHAC,HtAAC,HAST,AAST,HAS_y,HAC_y,AAS_y,AAC_y
Team,,,,,,,,,,,,,,
Arsenal,0.46875,0.37500,0.233333,0.466667,0.352941,0.083333,0.666667,0.5000,0.626866,0.218182,0.538462,0.75,1.0,0.000000
Aston Villa,0.43750,0.31250,0.600000,0.433333,0.352941,0.750000,0.222222,0.6250,0.208955,0.454545,0.153846,0.50,0.2,0.333333
Bournemouth,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.076923,0.50,0.4,1.000000
Brentford,0.25000,0.40625,0.400000,0.700000,0.000000,0.583333,0.333333,0.9375,0.238806,0.454545,0.692308,0.50,0.8,0.250000
Brighton,0.00000,0.31250,0.433333,0.300000,0.117647,0.666667,0.166667,0.5000,0.000000,0.581818,0.384615,0.25,0.8,0.000000
Chelsea,0.59375,0.84375,0.233333,0.033333,0.470588,0.000000,0.833333,0.1875,0.567164,0.218182,0.384615,0.75,0.2,0.166667
Crystal Palace,0.46875,0.31250,0.333333,0.500000,0.294118,0.583333,0.333333,0.7500,0.223881,0.345455,0.230769,0.75,0.4,0.166667
Everton,0.31250,0.15625,0.400000,0.833333,0.117647,0.416667,0.166667,0.8750,0.134328,0.454545,0.000000,0.25,0.4,0.083333
Fulham,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.461538,1.00,0.2,0.083333


In [26]:
# Since we don't have data on Half Time stats and average shots on target for the newly promoted teams,
# we will want to drop these columns.
curr_team_data.drop(['HtHAS','HtAAS', 'HtHAC', 'HtAAC', 'HAST', 'AAST'],axis=1, inplace=True)
curr_team_data

,HAS_x,AAS_x,HAC_x,AAC_x,HAS_y,HAC_y,AAS_y,AAC_y
Team,,,,,,,,
Arsenal,0.46875,0.37500,0.233333,0.466667,0.538462,0.75,1.0,0.000000
Aston Villa,0.43750,0.31250,0.600000,0.433333,0.153846,0.50,0.2,0.333333
Bournemouth,NaN,NaN,NaN,NaN,0.076923,0.50,0.4,1.000000
Brentford,0.25000,0.40625,0.400000,0.700000,0.692308,0.50,0.8,0.250000
Brighton,0.00000,0.31250,0.433333,0.300000,0.384615,0.25,0.8,0.000000
Burnley,0.09375,0.12500,0.433333,0.533333,NaN,NaN,NaN,NaN
Chelsea,0.59375,0.84375,0.233333,0.033333,0.384615,0.75,0.2,0.166667
Crystal Palace,0.46875,0.31250,0.333333,0.500000,0.230769,0.75,0.4,0.166667
Everton,0.31250,0.15625,0.400000,0.833333,0.000000,0.25,0.4,0.083333


In [27]:
# Sum the averages for HAS
has = ['HAS_x','HAS_y']
curr_team_data['HAS'] = curr_team_data[has].mean(axis=1)
curr_team_data.drop(has,axis=1, inplace=True)

# Sum the averages for HAC
hac = ['HAC_x','HAC_y']
curr_team_data['HAC'] = curr_team_data[hac].mean(axis=1)
curr_team_data.drop(hac,axis=1, inplace=True)

# Sum the averages for AAS
aas = ['AAS_x','AAS_y']
curr_team_data['AAS'] = curr_team_data[aas].mean(axis=1)
curr_team_data.drop(aas,axis=1, inplace=True)

# Sum the averages for AAC
aac = ['AAC_x','AAC_y']
curr_team_data['AAC'] = curr_team_data[aac].mean(axis=1)
curr_team_data.drop(aac,axis=1, inplace=True)

In [28]:
# Calculate attacking strength for Home and Away
curr_team_data['HAtS'] = curr_team_data['HAS'] * curr_team_data['AAC']; # Home Attack Strength
curr_team_data['AAtS'] = curr_team_data['HAC'] * curr_team_data['AAS']; # Away Attack Strength

In [29]:
# Check the dataframe
curr_team_data

,HAS,HAC,AAS,AAC,HAtS,AAtS
Team,,,,,,
Arsenal,0.503606,0.491667,0.687500,0.233333,0.117508,0.338021
Aston Villa,0.295673,0.550000,0.256250,0.383333,0.113341,0.140937
Bournemouth,0.076923,0.500000,0.400000,1.000000,0.076923,0.200000
Brentford,0.471154,0.450000,0.603125,0.475000,0.223798,0.271406
Brighton,0.192308,0.341667,0.556250,0.150000,0.028846,0.190052
Burnley,0.093750,0.433333,0.125000,0.533333,0.050000,0.054167
Chelsea,0.489183,0.491667,0.521875,0.100000,0.048918,0.256589
Crystal Palace,0.349760,0.541667,0.356250,0.333333,0.116587,0.192969
Everton,0.156250,0.325000,0.278125,0.458333,0.071615,0.090391


In [30]:
# Create the new season table with the averages and strengths for the home and away teams
curr_fixtures = new_season_df[['Home Team', 'Away Team']]


curr_fixtures = curr_fixtures.merge(curr_team_data['HAS'], left_on='Home Team', right_index=True, how='left')
curr_fixtures = curr_fixtures.merge(curr_team_data['HAC'], left_on='Home Team', right_index=True, how='left') 
curr_fixtures = curr_fixtures.merge(curr_team_data['AAS'], left_on='Away Team', right_index=True, how='left') 
curr_fixtures = curr_fixtures.merge(curr_team_data['AAC'], left_on='Away Team', right_index=True, how='left') 
curr_fixtures = curr_fixtures.merge(curr_team_data['HAtS'], left_on='Home Team', right_index=True, how='left') 
curr_fixtures = curr_fixtures.merge(curr_team_data['AAtS'], left_on='Away Team', right_index=True, how='left') 



fixture_list = curr_fixtures
curr_fixtures



,Home Team,Away Team,HAS,HAC,AAS,AAC,HAtS,AAtS
0,Crystal Palace,Arsenal,0.349760,0.541667,0.687500,0.233333,0.116587,0.338021
1,Fulham,Liverpool,0.461538,1.000000,0.700000,0.158333,0.038462,0.087500
2,Bournemouth,Aston Villa,0.076923,0.500000,0.256250,0.383333,0.076923,0.140937
3,Leeds,Wolves,0.317308,0.508333,0.109375,0.066667,0.224760,0.016406
4,Newcastle,Nottingham Forest,0.325721,0.550000,0.000000,0.500000,0.114002,0.000000
...,...,...,...,...,...,...,...,...
375,Everton,Bournemouth,0.156250,0.325000,0.400000,1.000000,0.071615,0.200000
376,Leeds,Tottenham,0.317308,0.508333,0.665625,0.200000,0.224760,0.194141
377,Leicester,West Ham,0.326923,0.716667,0.318750,0.216667,0.217949,0.233750
378,Man United,Fulham,0.426683,0.575000,0.200000,0.083333,0.124449,0.200000


In [31]:
# Get rid of non-numerical data so we can properly run the dataframe through our model
col_list = [x for x in col_list if x not in ['Result']]
curr_fixtures = curr_fixtures[col_list]
curr_fixtures

,HAS,HAC,AAS,AAC,HAtS,AAtS
0,0.349760,0.541667,0.687500,0.233333,0.116587,0.338021
1,0.461538,1.000000,0.700000,0.158333,0.038462,0.087500
2,0.076923,0.500000,0.256250,0.383333,0.076923,0.140937
3,0.317308,0.508333,0.109375,0.066667,0.224760,0.016406
4,0.325721,0.550000,0.000000,0.500000,0.114002,0.000000
...,...,...,...,...,...,...
375,0.156250,0.325000,0.400000,1.000000,0.071615,0.200000
376,0.317308,0.508333,0.665625,0.200000,0.224760,0.194141
377,0.326923,0.716667,0.318750,0.216667,0.217949,0.233750
378,0.426683,0.575000,0.200000,0.083333,0.124449,0.200000


In [32]:
# Run a prediction of this year's results with our new season's dataframe
predicted = model.predict(curr_fixtures)
predicted
predictedDF = pd.DataFrame(predicted, columns=['Predicted'])
predictedDF.head()

,Predicted
0,-1
1,-1
2,-1
3,1
4,1


In [33]:
# Create the full season table with the predicted results of each game.
fixture_list = fixture_list.merge(predictedDF, left_index=True, right_index=True)
fixture_list

,Home Team,Away Team,HAS,HAC,AAS,AAC,HAtS,AAtS,Predicted
0,Crystal Palace,Arsenal,0.349760,0.541667,0.687500,0.233333,0.116587,0.338021,-1
1,Fulham,Liverpool,0.461538,1.000000,0.700000,0.158333,0.038462,0.087500,-1
2,Bournemouth,Aston Villa,0.076923,0.500000,0.256250,0.383333,0.076923,0.140937,-1
3,Leeds,Wolves,0.317308,0.508333,0.109375,0.066667,0.224760,0.016406,1
4,Newcastle,Nottingham Forest,0.325721,0.550000,0.000000,0.500000,0.114002,0.000000,1
...,...,...,...,...,...,...,...,...,...
375,Everton,Bournemouth,0.156250,0.325000,0.400000,1.000000,0.071615,0.200000,1
376,Leeds,Tottenham,0.317308,0.508333,0.665625,0.200000,0.224760,0.194141,0
377,Leicester,West Ham,0.326923,0.716667,0.318750,0.216667,0.217949,0.233750,0
378,Man United,Fulham,0.426683,0.575000,0.200000,0.083333,0.124449,0.200000,1


In [34]:
# Set up our final table.
final_predicted_table= pd.DataFrame(index=fixture_list['Home Team'].unique())
final_predicted_table['Predicted Points'] = 0

In [35]:
# Calculate the total points form the number of wins, draws and losses.
final_predicted_table['Predicted Points'] = 0 

def make_actual_table(x, ht, at, which):
    if x == 1:
        final_predicted_table.loc[ht, which] += 3
    elif x == 0:
        final_predicted_table.loc[ht, which] += 1
        final_predicted_table.loc[at, which] += 1
    elif x == -1:
        final_predicted_table.loc[at, which] += 3

fixture_list.apply(lambda x: make_actual_table(x.Predicted, x['Home Team'], x['Away Team'], 'Predicted Points'), axis=1)


0      None
1      None
2      None
3      None
4      None
       ... 
375    None
376    None
377    None
378    None
379    None
Length: 380, dtype: object

In [36]:
final_predicted_table.sort_values('Predicted Points', ascending=0)

,Predicted Points
Leicester,91
Arsenal,88
Brentford,83
Liverpool,81
Man City,72
West Ham,70
Chelsea,69
Leeds,62
Crystal Palace,57
Man United,55
